In [ ]:
import pandas as pd
from plotnine import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import Model, Sequential
%matplotlib inline

## Load data

In [ ]:
import os
os.system('kaggle datasets download -d uciml/pima-indians-diabetes-database -f diabetes.csv');

In [ ]:
dataset_df = pd.read_csv('diabetes.csv')
dataset_df.head()

## Preprocess data

In [ ]:
means = dataset_df.mean(axis=0)
# dataset_df = dataset_df - dataset_df.mean(axis=0)
dataset_df /= dataset_df.max(axis=0)

## Select heathy people data

In [ ]:
df_healthy = dataset_df[dataset_df.Outcome <= 0]

In [ ]:
df_healthy.head()

In [ ]:
REMOVE_FEATURES = ['BMI', 'Outcome']
FEATURES = [feature for feature in df_healthy.columns if feature not in REMOVE_FEATURES]
N_FEATURES = len(FEATURES)

In [ ]:
x_train = df_healthy[FEATURES].values
y_train = df_healthy[[REMOVE_FEATURES[0]]].values

## Create an ensemble of models

In [ ]:
def create_base_model(n_hidden):
    i = Input((N_FEATURES, ))
    h = Dense(n_hidden, kernel_initializer='normal', use_bias=True)(i)
    h = LeakyReLU()(h)
#     h = Dense(n_hidden, kernel_initializer='normal', activation='relu', use_bias=False)(h)
    o = Dense(1)(h)
    return Model(i, o)

In [ ]:
models = []
N_MODELS = 5
n_hidden_neurons = 16
for _ in range(N_MODELS):
    models.append(create_base_model(n_hidden_neurons))

In [ ]:
models[0].summary()

## Train ensemble

In [ ]:
# def train_model(model, input, target, n_epochs, lr=0.2):
# #     model.compile(optimizer=keras.optimizers.RMSprop(lr=0.02), loss='mean_squared_error', metrics=['mae'])
#     model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss='mean_squared_error', metrics=['mae'])
# #     model.fit(input, target, epochs=150)
#     return Model.fit(input, target, epochs=n_epochs, validation_split=0.2)

def train(model, x, y, lr, batch_size, epochs=1, history=None):
    current_epoch = 0 if history is None else len(history.history['loss'])
    model.compile(
        loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(lr=lr),
        metrics=['mae']
    )
    
    new_history = model.fit(
        x, y, epochs=current_epoch+epochs,
        batch_size=batch_size,
        initial_epoch=current_epoch,
        verbose=1
    )
    
    if history is not None:
        for key in new_history.history:
            history.history[key].extend(new_history.history[key])
    else:
        history = new_history
    
    return history

In [ ]:
from collections import defaultdict
histories = defaultdict(lambda:None)
for lr in [0.1]:
    for i in range(N_MODELS):
        histories[i] = train(models[i], x_train, y_train, lr, 200, history=histories[i])

In [ ]:
models[0].predict(x_train)

In [ ]:
y_train_pred = models[1].predict(x_train).flatten()

plt.scatter(y_train, y_train_pred)
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())

Done!